In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
import pickle
import string
import json
from os.path import exists

C:\Users\Josh\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Josh\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\Josh\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [21]:
#Using a sample size of 1000 for testing purposes
df = pd.read_csv("C:\\Users\\Josh\\Downloads\\project_1_Wiki_sample.csv")

In [24]:
df['id'][0]

1

In [3]:
#Initializing preproccesing tools and a empty dictionary to store processed data
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
stop_words.add(".")
stop_words.add(",")
processed_collection = {}

In [ ]:
#The following code processes the contents of each document and saves the information into the variable processed_collection
#Processed_collection is formatted {document_id: {title: 'document_title', content: "document_content"}}
for chunk in tqdm(pd.read_csv("D:\Downloads\project_1_Wiki_sample.csv", chunksize=1000)):
    for index, row in chunk.iterrows():
        word_tokens = word_tokenize(row['content'])
        word_tokens = set(word_tokens)
        word_tokens = word_tokens -stop_words
        word_tokens = [lemmatizer.lemmatize(x.lower()) for x in word_tokens]        
        processed_collection[row['id']] = {"content":word_tokens}

In [ ]:
try:
    geeky_file = open('processed', 'wb')
    pickle.dump(processed_collection, geeky_file)
    geeky_file.close()
  
except:
    print("Something went wrong")

In [ ]:
#The following code processes the contents of each document and saves the information into the variable processed_collection
#Processed_collection is formatted {document_id: {title: 'document_title', content: "document_content"}}
for index, row in df.iterrows():
    word_tokens = word_tokenize(row['content'])
    word_tokens = set(word_tokens)
    word_tokens = word_tokens - stop_words
    word_tokens = [lemmatizer.lemmatize(x.lower()) for x in word_tokens]   

    processed_collection[row['id']] = {"content":word_tokens}

In [ ]:
#The following code displays an example of a documents content after being processed 
processed_collection[1]['content']

In [ ]:
#The following function is used to created an inverted
#The parameter 'collection' is a dictionary that contains a nested dictionary where one of the keys must be "content"
def create_inverted_index(collection):
    inverted_index = {}
    for doc in tqdm(collection):
        for word in collection[doc]['content']:
            if word not in inverted_index:
                inverted_index[word] = [[doc,[lemmatizer.lemmatize(x.lower()) for x in df['content'][doc-1].split()].count(word)]
            else:
                inverted_index[word].append([doc, [lemmatizer.lemmatize(x.lower()) for x in df['content'][doc-1].split()].count(word)])
    return inverted_index

In [ ]:
file_to_read = open("processed", "rb")
processed_collection = pickle.load(file_to_read)

In [ ]:
#The inverted_index created from the processed collection
inverted_index = create_inverted_index(processed_collection)

In [ ]:
try:
    geeky_file2 = open("inverted_index", 'wb')
    pickle.dump(inverted_index, geeky_file2)
    geeky_file2.close()
except:
    print("Something went wrong")

In [2]:
file_to_read = open("inverted_index", "rb")
inverted_index = pickle.load(file_to_read)

In [5]:
#The following is just sample data for testing the function in the cell below
query = "fried chicken"
tfidf_1 = {"fried": [1 , 2 ,4, 8, 10], "chicken": [1,4,8,9, 10, 12]}

In [5]:
import itertools
import collections
def getResults(query, tfidf):
    query = [lemmatizer.lemmatize(x.lower()) for x  in word_tokenize(query)]
    list_of_documents = []
    for token in query:
        if token in tfidf.keys():
            list_of_documents.append(set(tfidf[token]))
#     list_of_documents = set(itertools.chain.from_iterable(list_of_documents))
    list_of_documents = list(set.intersection(*map(set,list_of_documents)))
    
    unchecked_tokens = query
    while len(list_of_documents) < 6 and not(len(unchecked_tokens) == 0):
        d = {}
        for token in unchecked_tokens:
            d[token] = len(tfidf[token])
        lowest = min(d, key=d.get)
        unchecked_tokens.pop(list(d.keys()).index(lowest))
        list_of_documents = list(set(list_of_documents).union(set(tfidf[lowest])))
    return list_of_documents, query    
            
    

In [65]:
list_of_cannidates, clean_query = getResults(query,tfidf_1)

AttributeError: 'dict' object has no attribute 'has_key'

In [6]:
def ranking(clean_query, list_of_cannidates, tfidf):
    total_num_of_docs = len(df)
    term_probs = {}
    for doc in list_of_cannidates:
        term_probs[doc] = 1
        for term in clean_query:
            temp_list = [lemmatizer.lemmatize(x.lower()) for x in word_tokenize(df['content'][doc-1])]
            if temp_list.count(term) == 0:
                term_probs[doc] = term_probs[doc] * 0.00001 * ((0.00001) / total_num_of_docs)
            else:
                term_probs[doc] = term_probs[doc] * (temp_list.count(term) / len(temp_list)) * (len(tfidf[term]) / total_num_of_docs) 
    return term_probs

In [7]:
from flask import Flask, render_template,request, redirect, request

In [22]:
df2 = pd.read_csv("C:\\Users\\Josh\\Desktop\\cs437p1\\project_1_AOL_query_log\\Clean-Data-01.txt",sep='\t')
df2 = df2.append(pd.read_csv("C:\\Users\\Josh\\Desktop\\cs437p1\\project_1_AOL_query_log\\Clean-Data-02.txt",sep='\t'))
df2 = df2.append(pd.read_csv("C:\\Users\\Josh\\Desktop\\cs437p1\\project_1_AOL_query_log\\Clean-Data-03.txt",sep='\t'))
df2 = df2.append(pd.read_csv("C:\\Users\\Josh\\Desktop\\cs437p1\\project_1_AOL_query_log\\Clean-Data-04.txt",sep='\t'))
df2 = df2.append(pd.read_csv("C:\\Users\\Josh\\Desktop\\cs437p1\\project_1_AOL_query_log\\Clean-Data-05.txt",sep='\t'))
df2["Query"] = df2["Query"].astype(str)
df2["QueryTime"]= pd.to_datetime(df2["QueryTime"])

In [24]:
app = Flask(__name__)
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

@app.route('/')
def home():
    return render_template("index.html")

@app.route('/ajax', methods = ['POST'])
def returnHome():
    print("homeeee")
    return redirect("index.html")

@app.route('/suggest', methods = ['POST'])
def suggest():
    """Returns new df with queries that start with the query parameter. Sorted by frequency, excludes exact match."""
    query = request.form['myQuery']
    sg = df2.loc[df2['Query'].str.startswith(query)]
    sg = sg[sg['Query'] != query]
    sg = sg.groupby(['Query']).size().reset_index(name='freq')
    sg = sg.sort_values(by=['freq'],ascending=False)
    data_dict = sg.head(5).to_dict()
    return data_dict['Query']

def build_results(data):
    sorted_dict = dict(sorted(data.items(), key=lambda item: item[1], reverse=True))
#     sorted_dict = dict(itertools.islice(data.items(), 10))
    results = {}
    for key in sorted_dict:
        results[key] = {"content":df['content'][key-1], "title":df['title'][key-1]}
    return results

@app.route('/results')
def results(query):
    print("resultsss", query)
    list_of_cannidates, clean_query = getResults(query,inverted_index)
    unsorted_dictionary_of_doc_probs = ranking(clean_query, list_of_cannidates, inverted_index)
#     sorted_dict = dict(sorted(unsorted_dictionary_of_doc_probs.items(), key=lambda item: item[1]))
#     sorted_dict = dict(itertools.islice(sorted_dict.items(), 10))
#     print(sorted_dict)
    results = build_results(unsorted_dictionary_of_doc_probs)
    # results = {1: {"content":"pizza","title":"place to eat" }, 2:{"content":"hamburger", "title": "food"}}    
    return render_template("results.html", query=clean_query, results = results, number_of_results = len(results))

@app.route('/', methods=['POST'])
def getQuery():
    query = request.form['myQuery']
    return results(query)
    # return render_template("results.html")
    
if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5505)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5505/ (Press CTRL+C to quit)
192.168.0.89 - - [16/Oct/2021 01:38:20] "GET / HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:38:20] "GET /static/index.js HTTP/1.1" 304 -
192.168.0.89 - - [16/Oct/2021 01:38:20] "GET /static/Search.gif HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:38:37] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:38:37] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:38:39] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:38:39] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:38:39] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:38:44] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:38:49] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:38:55] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:38:55] "POST /suggest HTTP/1.1" 200 -


resultsss pizza dough


192.168.0.89 - - [16/Oct/2021 01:38:56] "POST / HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:38:58] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:41:19] "GET / HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:41:27] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:41:30] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:41:55] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:42:06] "GET / HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:42:17] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:42:17] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:44:10] "GET / HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:44:19] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:44:19] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:44:19] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:44:34] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:44:4

resultsss 

...

ham glaze

" 200 -


192.168.0.89 - - [16/Oct/2021 01:46:03] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:46:03] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:46:03] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:46:03] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:46:03] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:46:05] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:46:05] "POST /suggest HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:46:06] "POST / HTTP/1.1" 200 -
192.168.0.89 - - [16/Oct/2021 01:46:12] "POST /suggest HTTP/1.1" 200 -


resultsss c\


[2021-10-16 01:46:12,820] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\Josh\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Josh\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Josh\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Josh\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Josh\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Josh\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-24-9520d28423cc>", line 51, in getQuery
    return results(query)
  File